In [1]:
import jax.numpy as np
from jax import grad, jit, vmap
from jax import random, api
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

from svgd import SVGD, get_bandwidth

In [2]:
from jax.scipy.stats import norm

@jit
def logp(x):
    """
    IN: single scalar np array x. alternatively, [x] works too
    OUT: scalar logp(x)
    """
    return np.squeeze(np.sum(norm.logpdf(x, loc=0, scale=1)))

In [3]:
n = 400
d = 1
stepsize = 0.01
L = int(10 / stepsize)
svgd_adaptive = SVGD(logp, n_iter_max=L, adaptive_kernel=True, get_bandwidth=get_bandwidth)

In [4]:
# generate data
rkey = random.PRNGKey(0)
x0 = random.normal(rkey, (n,1)) - 10

/home/lauro/.virtualenvs/msc-thesis/lib/python3.6/site-packages/jax/lib/xla_bridge.py:123: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [5]:
xout, log = svgd_adaptive.unjitted_svgd(x0, stepsize, bandwidth=0, n_iter=L)
xout.block_until_ready()
print()

In [6]:
st = time.time()
xout, log = svgd_adaptive.svgd(x0, stepsize, bandwidth=0, n_iter=L)
e1 = time.time()
xout.block_until_ready()
e2 = time.time()
print("until before block:", e1 - st, "s")
print("until after block:", e2 - st, "s")

COMPILING
until before block: 8.678895235061646 s
until after block: 148.08810424804688 s


# compare `jit`ted and non`jit`ted times

In [7]:
m = 1
generate_new_data = True

def sample_q0(rkey):
    rkey = random.split(rkey)[0]
    x0 = random.normal(rkey, (n, d))
    return x0, rkey

## Using jit

In [ ]:
times_j = []
rkey = random.PRNGKey(0)

for _ in tqdm(range(m)):
    if generate_new_data:
        x0, rkey = sample_q0(rkey)
    
    st = time.time()
    xout, log = svgd_adaptive.svgd(x0, stepsize, bandwidth=0, n_iter=L)
    xout = xout.block_until_ready()
    end = time.time()
    times_j.append(end - st)
times_j = np.array(times_j)
print("mean time:", np.mean(times_j), "s")
print("stdev:", np.std(times_j), "s")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
_ = plt.hist(times_j)

## Without jit

In [ ]:
times_nojit1 = []
rkey = random.PRNGKey(0)

with api.disable_jit():
    for _ in tqdm(range(m)):
        if generate_new_data:
            x0, rkey = sample_q0(rkey)
        
        st = time.time()
        xout, log = svgd_adaptive.svgd(x0, stepsize, bandwidth=0, n_iter=L)
#         xout = xout.block_until_ready()
        end = time.time()
        times_nojit1.append(end - st)
times_nojit1 = np.array(times_nojit1)
print("mean time:", np.mean(times_nojit1), "s")
print("stdev:", np.std(times_nojit1), "s")

In [ ]:
_ = plt.hist(times_nojit1)

In [ ]:
times_nojit2 = []
rkey = random.PRNGKey(0)

for _ in tqdm(range(m)):
    if generate_new_data:
        x0, rkey = sample_q0(rkey)

    st = time.time()
    xout, log = svgd_adaptive.unjitted_svgd(x0, stepsize, bandwidth=0, n_iter=L)
    end = time.time()
    times_nojit2.append(end - st)
times_nojit2 = np.array(times_nojit2)
print("mean time:", np.mean(times_nojit2), "s")
print("stdev:", np.std(times_nojit2), "s")

In [ ]:
svgd_adaptive.n_iter_max

In [ ]:
_ = plt.hist(times_nojit2)

# Final comparison

In [ ]:
cols = ["r", "b", "g"]
labels = ["Jitted", "Not jitted 1", "Not jitted 2"]

fig = plt.figure(1, figsize=[6.4, 4.8])
for i, times in enumerate([times_j, times_nojit1, times_nojit2]):
    plt.hist(times, color=cols[i], label=labels[i])
    
_ = plt.legend()